In [18]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import pickle
import copy


In [19]:
def save_intermediate_data(data, file_path):
    """
    Save intermediate data to a file using pickle serialization.

    Args:
        data: The data to be saved.
        file_path (str): The path of the file to save the data.
    """
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

def load_intermediate_data(file_path):
    """
    Load intermediate data from a file.

    Args:
        file_path (str): The path of the file to load the data from.

    Returns:
        The loaded data.
    """
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data


In [20]:
print(torch.hub.list("chenyaofo/pytorch-cifar-models", force_reload=True))

Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/zipball/master" to /home/chx/.cache/torch/hub/master.zip


['cifar100_mobilenetv2_x0_5', 'cifar100_mobilenetv2_x0_75', 'cifar100_mobilenetv2_x1_0', 'cifar100_mobilenetv2_x1_4', 'cifar100_repvgg_a0', 'cifar100_repvgg_a1', 'cifar100_repvgg_a2', 'cifar100_resnet20', 'cifar100_resnet32', 'cifar100_resnet44', 'cifar100_resnet56', 'cifar100_shufflenetv2_x0_5', 'cifar100_shufflenetv2_x1_0', 'cifar100_shufflenetv2_x1_5', 'cifar100_shufflenetv2_x2_0', 'cifar100_vgg11_bn', 'cifar100_vgg13_bn', 'cifar100_vgg16_bn', 'cifar100_vgg19_bn', 'cifar100_vit_b16', 'cifar100_vit_b32', 'cifar100_vit_h14', 'cifar100_vit_l16', 'cifar100_vit_l32', 'cifar10_mobilenetv2_x0_5', 'cifar10_mobilenetv2_x0_75', 'cifar10_mobilenetv2_x1_0', 'cifar10_mobilenetv2_x1_4', 'cifar10_repvgg_a0', 'cifar10_repvgg_a1', 'cifar10_repvgg_a2', 'cifar10_resnet20', 'cifar10_resnet32', 'cifar10_resnet44', 'cifar10_resnet56', 'cifar10_shufflenetv2_x0_5', 'cifar10_shufflenetv2_x1_0', 'cifar10_shufflenetv2_x1_5', 'cifar10_shufflenetv2_x2_0', 'cifar10_vgg11_bn', 'cifar10_vgg13_bn', 'cifar10_vgg16_b

In [21]:
!git clone --recursive https://github.com/chenyaofo/image-classification-codebase
%cd image-classification-codebase

Cloning into 'image-classification-codebase'...
remote: Enumerating objects: 875, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 875 (delta 51), reused 70 (delta 23), pack-reused 768
Receiving objects: 100% (875/875), 168.53 KiB | 2.03 MiB/s, done.
Resolving deltas: 100% (497/497), done.
Submodule 'codebase/torchutils' (https://github.com/chenyaofo/torchutils) registered for path 'codebase/torchutils'
Cloning into '/home/chx/Desktop/rsch/FairFace/image-classification-codebase/image-classification-codebase/codebase/torchutils'...
remote: Enumerating objects: 203, done.        
remote: Counting objects: 100% (203/203), done.        
remote: Compressing objects: 100% (143/143), done.        
remote: Total 203 (delta 120), reused 142 (delta 59), pack-reused 0        
Receiving objects: 100% (203/203), 56.04 KiB | 1.51 MiB/s, done.
Resolving deltas: 100% (120/120), done.
Submodule path 'codebase/torchutils': checked out '

In [22]:
%pip install -qr requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [23]:
from IPython.display import clear_output

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.0.0+cu117 CPU


## evaluate CIFAR10-pre-trained model on CIFAR-10

In [24]:
# !python -m entry.run --conf conf/cifar10.conf -o output/cifar10/vgg16_bn -M model.name=cifar10_vgg16_bn model.pretrained=true only_evaluate=true

In [25]:
# Define the transform to be applied to the test set
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

# Load the test set
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4,
                                         shuffle=False, num_workers=2)



100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data


In [26]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg16_bn", pretrained=True)

Using cache found in /home/chx/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


In [27]:
# Set the model to evaluation mode
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [28]:
# Move the model to the GPU device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [29]:
# Define the criterion
criterion = torch.nn.CrossEntropyLoss()

# Define variables to store the total number of correctly classified samples and total samples for each class
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

# Test the model on the test set
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# Print the accuracy for each class
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        test_set.classes[i], 100 * class_correct[i] / class_total[i]))

# Compute the overall accuracy
total_correct = sum(class_correct)
total_samples = sum(class_total)
accuracy = 100 * total_correct / total_samples

# Print the overall accuracy
print('Accuracy of the network on the test images: %d %%' % accuracy)

Accuracy of airplane : 95 %
Accuracy of automobile : 96 %
Accuracy of  bird : 92 %
Accuracy of   cat : 87 %
Accuracy of  deer : 94 %
Accuracy of   dog : 88 %
Accuracy of  frog : 96 %
Accuracy of horse : 96 %
Accuracy of  ship : 96 %
Accuracy of truck : 95 %
Accuracy of the network on the test images: 93 %


In [30]:
# Define class labels for CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


## look at activation levels

In [31]:

# load the model
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg16_bn", pretrained=True)
if torch.cuda.is_available():
    model.cuda()

# Define a function to be called by the hook

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    # acti = output.detach().cpu().numpy()
    # print(acti.shape)
    # fig, axs = plt.subplots(acti.shape[1], figsize=(5, 10))
    # for i in range(acti.shape[1]):
    #     axs[i].imshow(acti[0, i])
    # plt.show()
    return hook

for name, layer in model.named_modules():
    if isinstance(layer, nn.ReLU):
        print(name)
        layer.register_forward_hook(get_activation(name))
print(activation)

features.2
features.5
features.9
features.12
features.16
features.19
features.22
features.26
features.29
features.32
features.36
features.39
features.42
classifier.1
classifier.4
{}


Using cache found in /home/chx/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


In [32]:
# Define a function to unnormalize the images
def unnormalize(img):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = img.cpu().data.numpy().transpose((1, 2, 0))
    img = std * img + mean
    img = np.clip(img, 0, 1)
    return img


def plot_images_and_labels(images, labels):
    # Plot the images
    fig, axes = plt.subplots(nrows=1, ncols=len(images), figsize=(4, 2))
    for i in range(len(images)):
        axes.imshow(unnormalize(images[i]))
        axes.axis('off')
        axes.set_title('Label: {}'.format(classes[labels[i]]))
    plt.show()


def identify_significantly_smaller_values(values, z_score_threshold = -2):
    # Calculate the mean and standard deviation of the list
    mean = np.mean(values)
    std_dev = np.std(values)
    print("mean", mean, "std_dev", std_dev)
    # Calculate the z-score for each value in the list
    z_scores = (values - mean) / std_dev

    # Identify the values in the list that are significantly smaller
    significantly_smaller_values = [values[i] for i in range(len(values)) if z_scores[i] < z_score_threshold]

    # Return the significantly smaller values
    return significantly_smaller_values

# Move the model to the GPU device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Test the model on the test set with a batch size of 4
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
total_wrong = 0

eg_size = 20
num_test_images = 0
# Loop over the test set and print the activations for each layer
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        activation_sums = []
        images, labels = images.to(device), labels.to(device)
        # print("image shape: {}".format(images.shape))
        # plot_images_and_labels(images, labels)
        # Get the model predictions
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        c = (predicted == labels).squeeze()
        # batch_size = 1
        label = labels[i]
        class_correct[label] += c.item()
        class_total[label] += 1
        if predicted[i] != labels[i]:
            total_wrong += 1
            print("Wrong prediction: predicted = {}, actual = {}".format(predicted[i].item(), labels[i].item()))

        # for i in range(len(labels)):
        #     label = labels[i]
        #     class_correct[label] += c[i].item()
        #     class_total[label] += 1
        #     if predicted[i] != labels[i]:
        #         total_wrong += 1
        #         print("Wrong prediction: predicted = {}, actual = {}".format(predicted[i].item(), labels[i].item()))

        layer_name = "features.22"
        # Print the activations for each layer
        print("Activations for layer {}:".format(layer_name))
        layer_output = activation[layer_name]
        layer_viz = layer_output[0, :, :, :]
        layer_viz = layer_viz.cpu().data
        print(layer_viz.shape)
        img_idx = 0
        num_filters = layer_viz.shape[0]
        # fig, axes = plt.subplots(nrows=int(num_filters/8), ncols=8, figsize=(15, 30))
        for i in range(num_filters):
            act = torch.sum(layer_viz[i]).item()
            # print("activation value: {}".format(act))
            activation_sums.append(act)
            # row_idx = i // 8
            # col_idx = i % 8
            # axes[row_idx, col_idx].imshow(layer_viz[i], cmap='gray')
            # axes[row_idx, col_idx].axis('off')
            # axes[row_idx, col_idx].set_title('Filter {}'.format(i+1))
        # plt.show()

        print("num of filters = {}".format(i + 1))
        num_test_images += 1
        significantly_smaller_values = identify_significantly_smaller_values(activation_sums)
        print("significantly_smaller_values:", significantly_smaller_values)
        # if num_test_images == eg_size:
        #     break

IndexError: index 9 is out of bounds for dimension 0 with size 4

# final script, pretrained model on CIFAR10, test on CIFAR10

In [ ]:
# Define the transform to be applied to the test set
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

# Load the test set
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1,
                                         shuffle=False, num_workers=2)

model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg16_bn", pretrained=True)
# Move the model to the GPU device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Files already downloaded and verified


Using cache found in /home/jinyli/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [ ]:

# # Define a hook for all ReLU layers to compute the sum of activations of each filter
# activation = {}
# def relu_hook(module, input, output, layer_name):
#     activation[layer_name] = output.detach()
# # Register the hook for all ReLU layers
# for layer_name, layer in model.named_modules():
#     if isinstance(layer, torch.nn.ReLU):
#         layer.register_forward_hook(relu_hook, layer_name=layer_name)
num_filters = {}
activation = {}
filter_sums = {}
def get_activation(name):
    def hook(model, input, output):
        # Squeeze the tensor to remove the dimension with size 1
        squeezed_tensor = torch.squeeze(output.detach())
        activation[name] = squeezed_tensor
        # If squeezed_tensor has more than one dimension, flatten it
        if len(squeezed_tensor.shape) > 1:
            flattened_tensor = torch.flatten(squeezed_tensor, start_dim=1)
        else:
            flattened_tensor = squeezed_tensor
        # Sum along the second dimension if it exists, otherwise keep the tensor as is
        if flattened_tensor.dim() > 1:
            summed_tensor = flattened_tensor.sum(dim=1)
        else:
            summed_tensor = flattened_tensor
        # Convert the tensor to a list
        summed_list = summed_tensor.tolist()
        filter_sums[name] = summed_list
        num_filters[name] = len(summed_list)
    return hook

for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.ReLU):
        layer.register_forward_hook(get_activation(name))



In [35]:
# Define class labels for CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# Define variables to track accuracy and class-wise accuracy
correct = 0
total = 0
class_correct = [0] * 10
class_total = [0] * 10

image_predictation_result = {cl: {} for cl in range(len(classes))}
activation_all_images = []
filtersum_all_images = []
miss_predicated_images = []
# Set the model to evaluation mode
model.eval()
with torch.no_grad():
    # Test the model on the test set with a batch size of 1
    for i, (images, labels) in enumerate(test_loader):
        # Move the input data to the GPU
        images = images.cuda()

        # Forward pass through the model
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        # Update overall accuracy and class-wise accuracy
        total += labels.size(0)
        
        correct += (predicted == labels.cuda()).sum().item()
        class_correct[labels] += (predicted == labels.cuda()).sum().item()
        class_total[labels] += 1
        if (predicted == labels.cuda()).sum().item() == 0:
            miss_predicated_images.append(i)

        image_predictation_result[labels.item()][i] = predicted.item()
        # activation_all_images.append(copy.deepcopy(activation))
        filtersum_all_images.append(copy.deepcopy(filter_sums))
        # Print the predicted and ground truth labels if the image is predicted wrong
        if predicted != labels.cuda():
            print("Predicted label: {}, Ground truth: {}".format(predicted.item(), labels.item()))



4


TypeError: only integer tensors of a single element can be converted to an index

In [ ]:


# Print the overall accuracy and accuracy for each class
print('size of test set: {}, accuracy of the network on the test set: {:.2f}%'.format(total, 100 * correct / total))
for i in range(10):
    print('Accuracy of {} : {:.0f}% of {} images'.format(test_set.classes[i], 100 * class_correct[i] / class_total[i], class_total[i]))
print("miss_predicated_images num = {}".format(len(miss_predicated_images)))


size of test set: 10000, accuracy of the network on the test set: 93.94%
Accuracy of airplane : 96% of 1000 images
Accuracy of automobile : 97% of 1000 images
Accuracy of bird : 92% of 1000 images
Accuracy of cat : 87% of 1000 images
Accuracy of deer : 95% of 1000 images
Accuracy of dog : 88% of 1000 images
Accuracy of frog : 96% of 1000 images
Accuracy of horse : 97% of 1000 images
Accuracy of ship : 96% of 1000 images
Accuracy of truck : 95% of 1000 images
miss_predicated_images num = 606


In [ ]:
# print(activation_all_images[0]["features.22"][0].shape)
# # Flatten the tensor
# flattened_tensor = torch.flatten(activation_all_images[0]["features.22"][0], start_dim=1)
# print(flattened_tensor.shape)

In [ ]:
print(len(activation_all_images))
# print(len(activation_all_images[0]))
# print(len(activation_all_images[0]["features.22"][0]))
hooked_layer_names = list(activation.keys())
print(hooked_layer_names)
print(len(filtersum_all_images[0]))
print((filtersum_all_images[0]))

0
['features.2', 'features.5', 'features.9', 'features.12', 'features.16', 'features.19', 'features.22', 'features.26', 'features.29', 'features.32', 'features.36', 'features.39', 'features.42', 'classifier.1', 'classifier.4']
15
{'features.2': [1.4852211475372314, 235.8168487548828, 36.603519439697266, 108.89674377441406, 116.87852478027344, 87.73178100585938, 3.0563886165618896, 292.5098571777344, 175.85919189453125, 0.975274920463562, 138.64932250976562, 17.34107780456543, 20.722084045410156, 4.242427825927734, 0.0, 112.82270812988281, 214.7198486328125, 14.675559997558594, 83.36270141601562, 107.26823425292969, 10.56089973449707, 0.7800963521003723, 41.83653259277344, 2.134467124938965, 14.436492919921875, 140.80712890625, 26.243972778320312, 20.52385139465332, 17.805335998535156, 240.66653442382812, 198.3101043701172, 1.327854037284851, 31.620269775390625, 43.208648681640625, 226.2267608642578, 299.66387939453125, 23.635974884033203, 147.15753173828125, 13.177526473999023, 0.85724

In [ ]:
# save intermediate results
save_intermediate_data(activation_all_images, "activation_CIFAR10_VGG16.pkl")
save_intermediate_data(image_predictation_result, "image_predictation_result.pkl")
save_intermediate_data(miss_predicated_images, "miss_predicated_images.pkl")
save_intermediate_data(hooked_layer_names, "hooked_layer_names.pkl")
save_intermediate_data(filtersum_all_images, "filtersum_all_images.pkl")
save_intermediate_data(num_filters, "num_filters.pkl")


In [ ]:
activation_all_images = load_intermediate_data("activation_CIFAR10_VGG16.pkl")
image_predictation_result = load_intermediate_data("image_predictation_result.pkl")
miss_predicated_images = load_intermediate_data("miss_predicated_images.pkl")
hooked_layer_names = load_intermediate_data("hooked_layer_names.pkl")
filtersum_all_images = load_intermediate_data("filtersum_all_images.pkl")
num_filters = load_intermediate_data("num_filters.pkl")


In [ ]:
num_test_images = len(filtersum_all_images)
def find_outliers_zscore(data, threshold=1):
    """
    Find outliers in a list of values using z-score method.

    Args:
        data (list or numpy array): List of numeric values.
        threshold (float, optional): Z-score threshold for identifying outliers. Default is 3.

    Returns:
        list: List of outlier values.
    """
    # Convert data to numpy array
    data = np.array(data)
    if np.std(data) == 0:
        return []
    # Calculate z-scores
    z_scores = (data - np.mean(data)) / np.std(data)
    # Find outliers based on the threshold
    # outliers = data[np.abs(z_scores) > threshold]
    outliers = np.where(z_scores < -threshold)[0]
    return outliers

# def find_outliers_in_activation_for_layer(layer_name, threshold=2):
#     for image in range(num_test_images):
#         filter_id = 0
#         for filter in activation_all_images[image][layer_name][0]:
#             outliers = find_outliers_zscore(filter.cpu())
#             if len(outliers) > 0:
#                 print("image # {}, filter # {} has outliers {}".format(image, filter_id, outliers))
#             filter_id += 1



In [ ]:
# find outlier for all mis_predicated images:
# cat (3)
class_id = 3
all_image_class = image_predictation_result[class_id].keys()
wrong_images = [x for x in all_image_class if image_predictation_result[class_id][x] != class_id]
print("num of wrong images in class {} is {}".format(class_id, len(wrong_images)))


num of wrong images in class 3 is 130


In [ ]:
print(len(filtersum_all_images[0]["features.22"]))


256


In [ ]:

z_score_threshold = 1.0

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    for image in wrong_images:
        # print("image # {}".format(image))
        outliers = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        if len(outliers) > 0:
            print("image # {}, layer # {} has outliers {}".format(image, layer_name, outliers))




layer features.2
image # 739, layer # features.2 has outliers [14 40]
image # 845, layer # features.2 has outliers [13 14 40]
image # 2046, layer # features.2 has outliers [ 0 14 39 40]
image # 2779, layer # features.2 has outliers [ 9 14 40]
image # 2831, layer # features.2 has outliers [ 0 14 40 56]
image # 2940, layer # features.2 has outliers [ 0  9 13 14 17 23 26 39 40 41]
image # 3343, layer # features.2 has outliers [ 0  9 13 14 32 36 39 40]
image # 4581, layer # features.2 has outliers [14 40]
image # 6900, layer # features.2 has outliers [ 0 13 14 40]
image # 6912, layer # features.2 has outliers [ 0 14 40]
image # 7202, layer # features.2 has outliers [ 0 14 40]
image # 7384, layer # features.2 has outliers [ 0  9 13 14 17 32 36 39 40]
image # 7680, layer # features.2 has outliers [ 0 13 14 24 40]
image # 8298, layer # features.2 has outliers [ 0  9 14 23 36 40]
image # 8580, layer # features.2 has outliers [13 14 40]
image # 8598, layer # features.2 has outliers [ 0  9 13 14

## activation level for the whole test set


In [ ]:

print(num_filters)

{'features.2': 64, 'features.5': 64, 'features.9': 128, 'features.12': 128, 'features.16': 256, 'features.19': 256, 'features.22': 256, 'features.26': 512, 'features.29': 512, 'features.32': 512, 'features.36': 512, 'features.39': 512, 'features.42': 512, 'classifier.1': 512, 'classifier.4': 512}


In [ ]:

# outliers for each layer for each image
z_score_threshold = 2
all_outliers_for_image = {x: {} for x in hooked_layer_names}
num_times_being_outlier = {}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    num_times_being_outlier[layer_name] = [0] * num_filters[layer_name]
    for image in range(num_test_images):
        all_outliers_for_image[layer_name][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        for k in all_outliers_for_image[layer_name][image]:
            num_times_being_outlier[layer_name][k] += 1


layer features.2
layer features.5
layer features.9
layer features.12
layer features.16
layer features.19
layer features.22
layer features.26
layer features.29
layer features.32
layer features.36
layer features.39
layer features.42
layer classifier.1
layer classifier.4


In [ ]:
non_activation_ratio_whole_data = {}
ratio_threshold = 0.01
for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    s = {x: num_times_being_outlier[layer_name][x] / num_test_images for x in range(num_filters[layer_name]) if num_times_being_outlier[layer_name][x] > 0}
    non_activation_ratio_whole_data[layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}
    # non_activation_ratio_whole_data[layer_name] = [x/num_test_images for x in num_times_being_outlier[layer_name]]



layer features.2
layer features.5
layer features.9
layer features.12
layer features.16
layer features.19
layer features.22
layer features.26
layer features.29
layer features.32
layer features.36
layer features.39
layer features.42
layer classifier.1
layer classifier.4


In [ ]:
for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_whole_data[layer_name])



features.2 {}
features.5 {}
features.9 {91: 0.2431, 105: 0.0948, 112: 0.0141, 114: 0.0946}
features.12 {}
features.16 {}
features.19 {}
features.22 {}
features.26 {}
features.29 {}
features.32 {}
features.36 {}
features.39 {}
features.42 {}
classifier.1 {}
classifier.4 {}


## non-activation times ratio for different classes, for right/wrong

In [ ]:

# outliers for each layer for each image
z_score_threshold = 2
all_outliers_for_miss_predicated_image = {x: {} for x in hooked_layer_names}
num_times_being_outlier_miss_predicated_images = {}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    num_times_being_outlier_miss_predicated_images[layer_name] = [0] * num_filters[layer_name]
    for image in wrong_images:
        all_outliers_for_miss_predicated_image[layer_name][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        for k in all_outliers_for_miss_predicated_image[layer_name][image]:
            num_times_being_outlier_miss_predicated_images[layer_name][k] += 1


layer features.2
layer features.5
layer features.9
layer features.12
layer features.16
layer features.19
layer features.22
layer features.26
layer features.29
layer features.32
layer features.36
layer features.39
layer features.42
layer classifier.1
layer classifier.4


In [ ]:
non_activation_ratio_miss_predicated_image = {}
num_miss_predicated_images = len(wrong_images)

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    s = {x: num_times_being_outlier_miss_predicated_images[layer_name][x] / num_miss_predicated_images for x in range(num_filters[layer_name]) if num_times_being_outlier_miss_predicated_images[layer_name][x] > 0}
    non_activation_ratio_miss_predicated_image[layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}


for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_miss_predicated_image[layer_name])



layer features.2
layer features.5
layer features.9
layer features.12
layer features.16
layer features.19
layer features.22
layer features.26
layer features.29
layer features.32
layer features.36
layer features.39
layer features.42
layer classifier.1
layer classifier.4
features.2 {}
features.5 {}
features.9 {48: 0.015384615384615385, 65: 0.015384615384615385, 91: 0.3076923076923077, 105: 0.06923076923076923, 112: 0.023076923076923078, 114: 0.13846153846153847}
features.12 {}
features.16 {}
features.19 {}
features.22 {}
features.26 {}
features.29 {}
features.32 {}
features.36 {}
features.39 {}
features.42 {}
classifier.1 {}
classifier.4 {9: 0.015384615384615385, 283: 0.015384615384615385}


In [ ]:
# print(num_times_being_outlier_miss_predicated_images["features.19"])


## non-activation times for different classes

In [ ]:
all_outliers_for_classes = {x: {y: {} for y in range(len(classes))} for x in hooked_layer_names}
num_times_being_outlier_classes = {y: {} for y in range(len(classes))}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    for cla in range(len(classes)):
        num_times_being_outlier_classes[cla][layer_name] = [0] * num_filters[layer_name]
        for image in image_predictation_result[cla]:
            all_outliers_for_classes[layer_name][cla][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
            for k in all_outliers_for_classes[layer_name][cla][image]:
                num_times_being_outlier_classes[cla][layer_name][k] += 1



layer features.2
layer features.5
layer features.9
layer features.12
layer features.16
layer features.19
layer features.22
layer features.26
layer features.29
layer features.32
layer features.36
layer features.39
layer features.42
layer classifier.1
layer classifier.4


In [ ]:
non_activation_ratio_classes = {y: {} for y in range(len(classes))}
ratio_threshold = 0.01
for cla in range(len(classes)):
    for layer_name in hooked_layer_names:
        print("layer {}".format(layer_name))
        s = {x: num_times_being_outlier_classes[cla][layer_name][x] / len(image_predictation_result[cla]) for x in range(num_filters[layer_name]) if num_times_being_outlier_classes[cla][layer_name][x] > 0}
        non_activation_ratio_classes[cla][layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}



layer features.2
layer features.5
layer features.9
layer features.12
layer features.16
layer features.19
layer features.22
layer features.26
layer features.29
layer features.32
layer features.36
layer features.39
layer features.42
layer classifier.1
layer classifier.4
layer features.2
layer features.5
layer features.9
layer features.12
layer features.16
layer features.19
layer features.22
layer features.26
layer features.29
layer features.32
layer features.36
layer features.39
layer features.42
layer classifier.1
layer classifier.4
layer features.2
layer features.5
layer features.9
layer features.12
layer features.16
layer features.19
layer features.22
layer features.26
layer features.29
layer features.32
layer features.36
layer features.39
layer features.42
layer classifier.1
layer classifier.4
layer features.2
layer features.5
layer features.9
layer features.12
layer features.16
layer features.19
layer features.22
layer features.26
layer features.29
layer features.32
layer features.3

In [ ]:
# for cla in range(len(classes)):
#     for layer_name in hooked_layer_names:
#         print(layer_name, non_activation_ratio_classes[cla][layer_name])

# less non-activation classes: 0, 2, 4, 8
# many: 1, 3, 5, 6, 7, 9
cla = 9
print('Accuracy of {} : {:.0f}% of {} images'.format(test_set.classes[cla], 100 * class_correct[cla] / class_total[cla], class_total[cla]))
for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_classes[cla][layer_name])


Accuracy of truck : 95% of 1000 images
features.2 {}
features.5 {1: 0.012, 33: 0.011}
features.9 {48: 0.031, 81: 0.014, 91: 0.512, 104: 0.025, 105: 0.226, 112: 0.035, 114: 0.125, 119: 0.021}
features.12 {33: 0.03, 94: 0.02}
features.16 {}
features.19 {}
features.22 {}
features.26 {}
features.29 {}
features.32 {}
features.36 {}
features.39 {}
features.42 {}
classifier.1 {}
classifier.4 {}



# final script, pretrainted model on imageNet1K from pytorch, test on CIFAR10


In [ ]:

# Define the transform to be applied to the test set
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

# Load the test set
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1,
                                         shuffle=False, num_workers=2)

model = torchvision.models.vgg16_bn(weights='VGG16_BN_Weights.DEFAULT')
# Move the model to the GPU device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Files already downloaded and verified


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [ ]:

# # Define a hook for all ReLU layers to compute the sum of activations of each filter
# activation = {}
# def relu_hook(module, input, output, layer_name):
#     activation[layer_name] = output.detach()
# # Register the hook for all ReLU layers
# for layer_name, layer in model.named_modules():
#     if isinstance(layer, torch.nn.ReLU):
#         layer.register_forward_hook(relu_hook, layer_name=layer_name)
num_filters = {}
activation = {}
filter_sums = {}
def get_activation(name):
    def hook(model, input, output):
        # Squeeze the tensor to remove the dimension with size 1
        squeezed_tensor = torch.squeeze(output.detach())
        activation[name] = squeezed_tensor
        # If squeezed_tensor has more than one dimension, flatten it
        if len(squeezed_tensor.shape) > 1:
            flattened_tensor = torch.flatten(squeezed_tensor, start_dim=1)
        else:
            flattened_tensor = squeezed_tensor
        # Sum along the second dimension if it exists, otherwise keep the tensor as is
        if flattened_tensor.dim() > 1:
            summed_tensor = flattened_tensor.sum(dim=1)
        else:
            summed_tensor = flattened_tensor
        # Convert the tensor to a list
        summed_list = summed_tensor.tolist()
        filter_sums[name] = summed_list
        num_filters[name] = len(summed_list)
    return hook

for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.ReLU):
        layer.register_forward_hook(get_activation(name))




In [ ]:
# Define class labels for CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# Define variables to track accuracy and class-wise accuracy
correct = 0
total = 0
class_correct = [0] * 10
class_total = [0] * 10

image_predictation_result = {cl: {} for cl in range(len(classes))}
activation_all_images = []
filtersum_all_images = []
miss_predicated_images = []
# Set the model to evaluation mode
model.eval()
with torch.no_grad():
    # Test the model on the test set with a batch size of 1
    for i, (images, labels) in enumerate(test_loader):
        # Move the input data to the GPU
        images = images.cuda()

        # Forward pass through the model
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        # Update overall accuracy and class-wise accuracy
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum().item()
        class_correct[labels] += (predicted == labels.cuda()).sum().item()
        class_total[labels] += 1
        if (predicted == labels.cuda()).sum().item() == 0:
            miss_predicated_images.append(i)

        image_predictation_result[labels.item()][i] = predicted.item()
        # activation_all_images.append(copy.deepcopy(activation))
        filtersum_all_images.append(copy.deepcopy(filter_sums))
        # Print the predicted and ground truth labels if the image is predicted wrong
        # if predicted != labels.cuda():
        #     print("Predicted label: {}, Ground truth: {}".format(predicted.item(), labels.item()))



In [ ]:


# Print the overall accuracy and accuracy for each class
print('size of test set: {}, accuracy of the network on the test set: {:.2f}%'.format(total, 100 * correct / total))
for i in range(10):
    print('Accuracy of {} : {:.0f}% of {} images'.format(test_set.classes[i], 100 * class_correct[i] / class_total[i], class_total[i]))
print("miss_predicated_images num = {}".format(len(miss_predicated_images)))



size of test set: 10000, accuracy of the network on the test set: 0.03%
Accuracy of airplane : 0% of 1000 images
Accuracy of automobile : 0% of 1000 images
Accuracy of bird : 0% of 1000 images
Accuracy of cat : 0% of 1000 images
Accuracy of deer : 0% of 1000 images
Accuracy of dog : 0% of 1000 images
Accuracy of frog : 0% of 1000 images
Accuracy of horse : 0% of 1000 images
Accuracy of ship : 0% of 1000 images
Accuracy of truck : 0% of 1000 images
miss_predicated_images num = 9997


In [ ]:
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy of the VGG16 model on the CIFAR-10 test set: {accuracy:.2f}%")


In [ ]:
accuracy = 100 * correct / total
print(f"Accuracy of the VGG16 model on the CIFAR-10 test set: {accuracy:.2f}%")


In [ ]:
print("hello world")

hello world


# how to visualize images

In [ ]:


def imshow(img):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = img.cpu().data.numpy().transpose((1, 2, 0))
    img = std * img + mean
    plt.imshow(np.transpose(img, (1, 2, 0)))
    plt.show()


dataiter = iter(test_loader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(1)))

In [ ]:
# Load the CIFAR10 dataset
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Choose some random images from the dataset
num_images = 5
indices = np.random.choice(range(len(train_dataset)), num_images)
images = [train_dataset[i][0] for i in indices]

# Display the images
fig, axes = plt.subplots(ncols=num_images, figsize=(10, 5))
for i in range(num_images):
    axes[i].imshow(np.transpose(images[i].numpy(), (1, 2, 0)))
    axes[i].axis('off')
plt.show()